# EEG signals to 2d-image by denosing using Wavelet transform



> Analyzing Electroencephalogram (EEG) signal is a challenge due to the various artifacts used by Electromyogram, eye blink and Electrooculogram. The present de-noising techniques that are based on the frequency selective filtering suffers from a substantial loss of the EEG data. Noise removal using wavelet has the characteristic of preserving signal uniqueness even if noise is going to be minimized. To remove noise from EEG signal, this research employed discrete wavelet transform. Root mean square difference has been used to find the usefulness of the noise elimination. In this research, four different discrete wavelet functions have been used to remove noise from the Electroencephalogram signal gotten from two different types of patients (healthy and epileptic) to show the effectiveness of DWT on EEG noise removal. The result shows that the WF orthogonal meyer is the best one for noise elimination from the EEG signal of epileptic subjects and the WF Daubechies 8 (db8) is the best one for noise elimination from the EEG signal on healthy subjects. 

Mamun, M., Al-Kadi, M., & Marufuzzaman, M. (2013). Effectiveness of Wavelet Denoising on Electroencephalogram Signals. Journal of Applied Research and Technology, 11(1). https://doi.org/10.1016/S1665-6423(13)71524-4

For more details, pleas visit [Effectiveness of Wavelet Denoising on Electroencephalogram Signals](https://www.sciencedirect.com/science/article/pii/S1665642313715244)

In [ ]:
import io
import matplotlib
matplotlib.use('agg')  # turn off interactive backend
import matplotlib.pyplot as plt
import numpy as np
import pywt
import pandas as pd


In [ ]:
# from this notebook: "Harmful Brain Activity Plotting eeg/spec(https://www.kaggle.com/code/ybbbat/harmful-brain-activity-plotting-eeg-spec)"
def plot_eeg(df, moving_avg=1):
    fig, axs = plt.subplots(21, 1, figsize=(7,3.2), sharex=True)
    for i, ax in enumerate(axs):
        if(i==5 or i==10 or i==15 or i==0 or i==20):
            ax.plot(0, color="white")
        else:
            ax.plot(df.iloc[:,i-1], color="black")
        ax.set_yticks([])
        ax.set_xticks([])
        ax.spines[["top", "bottom", "left", "right"]].set_visible(False)  
    with io.BytesIO() as buff:
        fig.savefig(buff, format='raw')
        buff.seek(0)
        plt.close()
        return((np.frombuffer(buff.getvalue(), dtype=np.uint8).reshape(320,700, 4))[32:-32,100:-88,0])

# Denoising function

In [ ]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):
    ret = {key:[] for key in x.columns}
    
    for pos in x.columns:
        coeff = pywt.wavedec(x[pos], wavelet, mode="per")
        sigma = (1/0.6745) * maddest(coeff[-level])

        uthresh = sigma * np.sqrt(2*np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

        ret[pos]=pywt.waverec(coeff, wavelet, mode='per')
    
    return pd.DataFrame(ret)

# List of mother Function you can use for wavelet transform

In [ ]:
# get eeg signal
df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")



("/kaggle/input/eeg-10-npy/eeg_10.npy")    
a np array containg index with a gap of atleast 10 seconds of eeg data

In [ ]:
indexr=np.load("/kaggle/input/eeg-10-npy/eeg_10.npy")

In [ ]:
df=df.drop(list(indexr))

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df

In [ ]:
from tqdm import tqdm


# coverting to 2d images

In [ ]:

index=[[0,4],[4,5],[5,6],[6,7],[0,0],[11,15],[15,16],[16,17],[17,18],[11,11],[0,1],[1,2],[2,3],[3,7],[7,7],[11,12],[12,13],[13,14],[14,18]]
a=0
import warnings
warnings.filterwarnings("ignore")
for k in tqdm(range(len(df))):
    a=np.array(df["eeg_id"])[k]
    b=int(np.array(df["eeg_label_offset_seconds"])[k])
    c=np.array(df["eeg_sub_id"])[k]
    eeg = pd.read_parquet("/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/"+str(a)+".parquet")
    eeg = eeg[int(20+b)*200:int(30+b)*200:10]
    banana_mon=pd.DataFrame()
    for i,j in index:
        banana_mon[eeg.columns[i]+'-'+eeg.columns[j]]=(eeg[eeg.columns[i]]-eeg[eeg.columns[j]])
    np.save(f"{a}_{c}.npy",abs((plot_eeg(denoise(-1*banana_mon, wavelet="db38"))).astype(np.int16)-255))
    if((k+1)%1000==0):
        print(k)